In [13]:
import cv2
import numpy as np
import math

In [ ]:
def rgb_to_yiq(img_rgb):
    img_yiq = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2YCrCb)
    return img_yiq

def increase_saturation(img, scale=1.5):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    s = cv2.multiply(s, scale)
    s = np.clip(s, 0, 255).astype(hsv_img.dtype)
    hsv_img = cv2.merge([h, s, v])
    img_saturated = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return img_saturated

def filter_color(img_yiq):
    y_channel, cr_channel, cb_channel = cv2.split(img_yiq)
    
    # 노란색 영역의 Cr, Cb 임계값 설정
    # Cr 빨강 청록 140, 170
    # Cb 파랑 노랑 100, 130
    # 노란색 210 (Y) 146 (Cr) 16 (Cb)
    lower_cr = 140
    upper_cr = 170
    lower_cb = 0
    upper_cb = 140

    
    # 밝기 채널의 임계값 설정 (어두운 영역 제외)
    lower_y = 30  # 최소 밝기값 (조정 가능)
    upper_y = 170 # 최대 밝기값
    
    # Cr, Cb 채널에서 임계값에 해당하는 영역 추출
    mask_cr = cv2.inRange(cr_channel, lower_cr, upper_cr)
    mask_cb = cv2.inRange(cb_channel, lower_cb, upper_cb)
    
    # Y 채널에서 임계값에 해당하는 영역 추출
    mask_y = cv2.inRange(y_channel, lower_y, upper_y)
    
    # Cr, Cb 임계값에 해당하는 마스크 생성
    color_mask_cr_cb = cv2.bitwise_and(mask_cr, mask_cb)
    
    # Y 임계값과 Cr, Cb 임계값을 결합
    color_mask = cv2.bitwise_and(color_mask_cr_cb, mask_y)
    
    return color_mask

In [ ]:
def gaussian_kernel(size, sigma):
    kernel = np.zeros((size, size), dtype=np.float32)
    center = size // 2
    for i in range(size):
        for j in range(size):
            diff = (i - center)**2 + (j - center)**2
            kernel[i, j] = np.exp(-diff / (2 * sigma**2))
    return kernel / np.sum(kernel)

def apply_gaussian_filter(image, kernel_size, sigma):
    kernel = gaussian_kernel(kernel_size, sigma)
    pad_size = kernel_size // 2
    padded_image = cv2.copyMakeBorder(image, pad_size, pad_size, pad_size, pad_size, cv2.BORDER_REFLECT_101)
    filtered_image = np.zeros_like(image)
    for c in range(image.shape[2]):
        filtered_image[:, :, c] = cv2.filter2D(padded_image[:, :, c], -1, kernel)[pad_size:-pad_size, pad_size:-pad_size]
    return filtered_image


def gaussian_filter_grayscale(input_img, size, sigma):  # 정규 분포를 따르며, 중심에서 멀어질수록 가중치가 작아진다.
    row, col = input_img.shape
    result_img = np.zeros((row, col))
    r = size // 2

    kernel = np.zeros((size, size))
    for x in range(-r, r+1):
        for y in range(-r, r+1):
            kernel[x+r][y+r] = (1 / (2 * math.pi * (sigma**2))) * math.exp(-(x**2 + y**2) / (2 * (sigma**2)))
            
    sum = 0.0
    for i in range(size):
        for j in range(size):
            sum += kernel[i][j]
            
    for i in range(size):   # 정규화
        for j in range(size):
            kernel[i][j] /= sum

    for i in range(r, row-r):
        for j in range(r, col-r):
            sum = 0.0
            for x in range(-r, r+1):
                for y in range(-r, r+1):
                    sum += input_img[i+x][j+y] * kernel[x+r][y+r]
            result_img[i][j] = round(sum)
            
    return result_img

In [ ]:
# 히스토그램 평활화 함수

def histo_equal(src):
    # Histogram의 bin 값 설정
    MAX_BIN = 256
    hist = np.zeros((MAX_BIN))  # 히스토그램 초기화
    hist_c = np.zeros((MAX_BIN))  # 누적 분포 함수 초기화
    # 히스토그램 계산
    for y in range(src.shape[0]):
        for x in range(src.shape[1]):
            hist[src[y][x]] += 1
    # 누적 분포 함수(CDF) 계산
    for i in range(1, MAX_BIN):
        hist_c[i] = hist_c[i-1] + hist[i]
    # 정규화
    hist_c /= (src.shape[0] * src.shape[1])
    # 평활화된 이미지 생성
    dst_hist = np.zeros((src.shape), dtype=np.uint8)
    for y in range(src.shape[0]):
        for x in range(src.shape[1]):
            dst_hist[y][x] = hist_c[src[y][x]] * 255
        
    return dst_hist

In [ ]:
def preprocess_image(frame):
    # 가우시안 블러링 적용
    # blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    blurred = apply_gaussian_filter(frame, 5, 1)
    
    # 히스토그램 평활화 적용 (HSI 색상 공간에서 적용)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v_eq = cv2.equalizeHist(v)
    #v_eq = histo_equal(v)
    hsv_eq = cv2.merge([h, s, v_eq])
    equalized = cv2.cvtColor(hsv_eq, cv2.COLOR_HSV2BGR)
    
    return equalized

In [22]:
def detect_circular_patterns(edges, frame):
    if edges is not None and len(edges.shape) == 2 and edges.dtype == np.uint8:
        # 원형 패턴 검출
        circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                   param1=50, param2=30, minRadius=10, maxRadius=30)
        
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                cv2.circle(frame, (x, y), r, (0, 0, 255), 4)  # 원의 중심에 빨간색 원 그리기
    return frame

In [23]:
def detect_linear_patterns(edges, frame):
    if edges is not None and len(edges.shape) == 2 and edges.dtype == np.uint8:
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 30, minLineLength=30, maxLineGap=20)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 4) 
    return frame

In [ ]:
# 동영상 파일 열기
cap = cv2.VideoCapture('./sample_video4.mp4')

# 동영상 파일이 없으면 에러 처리
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# 출력 동영상 파일 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', 
                      fourcc, 
                      cap.get(cv2.CAP_PROP_FPS), 
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 각 프레임에 대해 처리 반복
while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    # 전처리 (가우시안 필터링 및 히스토그램 평활화)
    preprocessed_frame = preprocess_image(frame)

    # H S 값 변환
    img_saturated = increase_saturation(preprocessed_frame, 1.2)

    # RGB 이미지를 YIQ 모델로 변환
    img_yiq = rgb_to_yiq(img_saturated)

    # YIQ 모델에서 색상 필터링
    color_mask = filter_color(img_yiq)  

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    #blurred = gaussian_filter_grayscale(gray, 5, 1)
    equalized = cv2.equalizeHist(blurred)
    if blurred.dtype != 'uint8':
        blurred = (blurred * 255).astype('uint8')
    edges = cv2.Canny(blurred, 100, 200)
    mask = cv2.bitwise_and(color_mask, edges)

    # 원형 점자블록 검출
    frame = detect_circular_patterns(mask, frame)
    
    # 선형 점자블록 검출
    frame = detect_linear_patterns(mask, frame)
    
    # 결과 프레임 출력
    cv2.imshow('Result', frame)
    
    # 결과 동영상 파일에 프레임 추가
    # mask_bgr = cv2.cvtColor(color_mask, cv2.COLOR_GRAY2BGR)
    out.write(frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 해제 작업
cap.release()
out.release()
cv2.destroyAllWindows()

2024-06-21 20:48:55.352 Python[56795:18304860] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
